# PennyLane Tutorial: Quantum Computing Fundamentals

## Introduction

**PennyLane** is a Python library for quantum machine learning, automatic differentiation, and optimization of hybrid quantum-classical computations. It seamlessly integrates classical machine learning libraries (PyTorch, TensorFlow, JAX) with quantum simulators and hardware.

### Key Features:
- **Device-agnostic**: Write once, run on any quantum simulator or hardware
- **Automatic differentiation**: Compute gradients of quantum circuits
- **Hybrid computations**: Mix quantum and classical processing
- **Optimization**: Built-in optimizers for variational algorithms

In [ ]:
!pip install pennylane -q

In [ ]:

# Import necessary libraries
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

print(f"PennyLane version: {qml.__version__}")

## 1. Quantum Devices

A **quantum device** represents the quantum hardware or simulator where computations are executed. PennyLane supports various backends:

- `default.qubit`: CPU-based simulator (good for small circuits)
- `lightning.qubit`: Fast C++ simulator
- `default.mixed`: Density matrix simulator (for noisy circuits)
- Hardware devices via plugins (IBM, Rigetti, IonQ, etc.)

Devices are initialized with the number of qubits (wires) they contain.

In [ ]:
# Create a quantum device with 2 qubits
dev = qml.device('default.qubit', wires=2)

print(f"Device: {dev.name}")
print(f"Number of wires: {dev.wires}")
print(f"Shots: {dev.shots}")  # None means exact expectation values

In [ ]:
qml.device?

## 2. Quantum Nodes (QNodes)

A **QNode** is the fundamental building block in PennyLane. It wraps a quantum function and connects it to a device. The quantum function defines:
1. **Quantum gates** to apply (the circuit)
2. **Measurements** to perform (what to return)

QNodes can be differentiated, optimized, and integrated into larger computational graphs.

### Basic Structure:
```python
@qml.qnode(device)
def quantum_function(params):
    # Apply quantum gates
    qml.Gate(params, wires=...)
    # Return measurement
    return qml.expval(qml.Observable(...))
```

In [ ]:
# Example 1: Simple single-qubit circuit
@qml.qnode(dev)
def simple_circuit(angle):
    """Rotate a qubit and measure in Z basis"""
    qml.RY(angle, wires=0)  # Rotation around Y-axis
    return qml.expval(qml.PauliZ(0))  # Expectation value of Z operator

# Test the circuit
angle = np.pi / 4
result = simple_circuit(angle)
print(f"Expectation value for angle π/4: {result:.4f}")

# Visualize how the expectation value changes with angle
angles = np.linspace(0, 2*np.pi, 100)
expectations = [simple_circuit(a) for a in angles]

plt.figure(figsize=(10, 4))
plt.plot(angles, expectations, linewidth=2)
plt.xlabel('Rotation Angle (radians)', fontsize=12)
plt.ylabel('⟨Z⟩', fontsize=12)
plt.title('Expectation Value vs Rotation Angle', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.show()

## 3. Common Quantum Gates

PennyLane provides a comprehensive set of quantum gates:

### Single-Qubit Gates:
- **Pauli gates**: `PauliX`, `PauliY`, `PauliZ` (bit-flip, phase-flip)
- **Hadamard**: `Hadamard` (superposition)
- **Rotations**: `RX`, `RY`, `RZ` (parameterized rotations)
- **Phase gate**: `PhaseShift` (adds relative phase)

### Multi-Qubit Gates:
- **CNOT**: `CNOT` (controlled-NOT, entangling gate)
- **CZ**: `CZ` (controlled-Z)
- **SWAP**: `SWAP` (swaps two qubits)
- **Toffoli**: `Toffoli` (controlled-controlled-NOT)

Let's explore these with examples:

In [ ]:
# Example 2: Creating a Bell state (maximally entangled state)
@qml.qnode(dev)
def bell_state():
    """Create the Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2"""
    qml.Hadamard(wires=0)      # Create superposition on qubit 0
    qml.CNOT(wires=[0, 1])     # Entangle qubit 1 with qubit 0
    return qml.state()          # Return the full quantum state

state = bell_state()
print("Bell State |Φ+⟩:")
print(f"State vector: {state}")
print(f"\nProbabilities: |00⟩={abs(state[0])**2:.3f}, |01⟩={abs(state[1])**2:.3f}, "
      f"|10⟩={abs(state[2])**2:.3f}, |11⟩={abs(state[3])**2:.3f}")

In [ ]:
# Example 3: Parameterized circuit with multiple gates
@qml.qnode(dev)
def parameterized_circuit(params):
    """
    A parameterized quantum circuit demonstrating various gates.
    params: array of shape (3,) containing rotation angles
    """
    # Apply rotations to first qubit
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    
    # Apply Hadamard to second qubit
    qml.Hadamard(wires=1)
    
    # Entangle the qubits
    qml.CNOT(wires=[0, 1])
    
    # Apply final rotation to second qubit
    qml.RZ(params[2], wires=1)
    
    # Measure both qubits in different bases
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliX(1))

# Test with random parameters
params = np.array([0.5, 1.0, 0.3])
result = parameterized_circuit(params)
print(f"Measurements with params {params}:")
print(f"⟨Z₀⟩ = {result[0]:.4f}")
print(f"⟨X₁⟩ = {result[1]:.4f}")

## 4. Measurements and Observables

PennyLane supports various measurement types:

### Measurement Functions:
- `qml.expval(observable)`: Expectation value ⟨O⟩
- `qml.var(observable)`: Variance var(O)
- `qml.probs(wires)`: Probability distribution
- `qml.sample(observable)`: Sample measurements
- `qml.state()`: Full quantum state vector

### Common Observables:
- Pauli operators: `PauliX`, `PauliY`, `PauliZ`
- Hermitian operators: `Hermitian`
- Tensor products: `qml.PauliZ(0) @ qml.PauliZ(1)`

In [ ]:
# Example 4: Different measurement types
dev_3q = qml.device('default.qubit', wires=3)

@qml.qnode(dev_3q)
def measurement_demo(angle):
    """Demonstrate various measurement types"""
    # Prepare a interesting state
    qml.RY(angle, wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(angle/2, wires=2)
    
    # Multiple measurements
    return (
        qml.expval(qml.PauliZ(0)),                    # Expectation value
        qml.var(qml.PauliX(1)),                       # Variance
        qml.probs(wires=[0, 1]),                      # Joint probability
        qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))    # Two-qubit observable
    )

expval, variance, probs, two_qubit = measurement_demo(np.pi/3)
print(f"Expectation ⟨Z₀⟩: {expval:.4f}")
print(f"Variance var(X₁): {variance:.4f}")
print(f"Joint probabilities P(00,01,10,11): {probs}")
print(f"Two-qubit correlation ⟨Z₀Z₁⟩: {two_qubit:.4f}")

## 5. Circuit Visualization

PennyLane provides tools to visualize quantum circuits, which is essential for understanding and debugging. The `qml.draw()` function or `qml.draw_mpl()` can display circuits in ASCII or as matplotlib figures.

In [ ]:
# Example 5: Visualizing circuits
@qml.qnode(dev)
def visualize_circuit(params):
    qml.RY(params[0], wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RZ(params[1], wires=1)
    qml.CNOT(wires=[1, 0])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

# ASCII drawing
print("Circuit diagram:")
print(qml.draw(visualize_circuit)([0.5, 0.3]))

# For matplotlib drawing (uncomment if you want graphical output):
fig, ax = qml.draw_mpl(visualize_circuit)([0.5, 0.3])
plt.show()

## 6. Automatic Differentiation

One of PennyLane's most powerful features is **automatic differentiation** of quantum circuits. This enables:
- Training quantum machine learning models
- Implementing variational quantum algorithms
- Gradient-based optimization

PennyLane computes gradients using various methods:
- **Parameter-shift rule**: Exact gradients for quantum circuits
- **Finite differences**: Numerical approximation
- **Backpropagation**: For simulators supporting it

Gradients are computed automatically using `qml.grad()` or integration with ML frameworks.

In [ ]:
# Example 6: Computing gradients
@qml.qnode(dev)
def differentiable_circuit(params):
    """A simple parameterized circuit"""
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RZ(params[2], wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

# Compute gradient
params = np.array([0.1, 0.2, 0.3], requires_grad=True)
gradient_fn = qml.grad(differentiable_circuit)
gradient = gradient_fn(params)

print(f"Parameters: {params}")
print(f"Function value: {differentiable_circuit(params):.4f}")
print(f"Gradient: {gradient}")

# Visualize the function and its gradient
param_range = np.linspace(-np.pi, np.pi, 50)
values = []
grads = []

for p in param_range:
    test_params = np.array([p, 0.2, 0.3], requires_grad=True)
    values.append(differentiable_circuit(test_params))
    grads.append(gradient_fn(test_params)[0])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

ax1.plot(param_range, values, linewidth=2)
ax1.set_xlabel('Parameter value', fontsize=12)
ax1.set_ylabel('⟨Z₀Z₁⟩', fontsize=12)
ax1.set_title('Function Value', fontsize=14)
ax1.grid(True, alpha=0.3)

ax2.plot(param_range, grads, linewidth=2, color='orange')
ax2.set_xlabel('Parameter value', fontsize=12)
ax2.set_ylabel('∂⟨Z₀Z₁⟩/∂θ₀', fontsize=12)
ax2.set_title('Gradient', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Optimization

PennyLane includes built-in optimizers for training quantum circuits:

- `GradientDescentOptimizer`: Basic gradient descent
- `AdamOptimizer`: Adaptive moment estimation
- `QNGOptimizer`: Quantum natural gradient
- `NesterovMomentumOptimizer`: Accelerated gradient descent

These optimizers work seamlessly with QNodes and automatically handle gradient computation.

In [ ]:
# Example 7: Optimization loop
# Goal: Find parameters that minimize the expectation value

@qml.qnode(dev)
def cost_function(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliX(1))

# Initialize optimizer and parameters
opt = qml.GradientDescentOptimizer(stepsize=0.1)
params = np.array([0.1, 0.1], requires_grad=True)

# Optimization loop
n_steps = 100
cost_history = []

print("Starting optimization...")
for step in range(n_steps):
    params, cost = opt.step_and_cost(cost_function, params)
    cost_history.append(cost)
    
    if step % 20 == 0:
        print(f"Step {step:3d}: Cost = {cost:.6f}")

print(f"\nFinal parameters: {params}")
print(f"Final cost: {cost:.6f}")

# Plot optimization progress
plt.figure(figsize=(10, 4))
plt.plot(cost_history, linewidth=2)
plt.xlabel('Optimization Step', fontsize=12)
plt.ylabel('Cost', fontsize=12)
plt.title('Optimization Progress', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

## 8. Templates and Circuit Layers

PennyLane provides **templates** - pre-built circuit architectures commonly used in quantum machine learning:

- `AngleEmbedding`: Encode classical data into rotation angles
- `AmplitudeEmbedding`: Encode data into quantum amplitudes
- `StronglyEntanglingLayers`: Variational circuit with entanglement
- `BasicEntanglerLayers`: Simple entangling layers
- `QAOAEmbedding`: QAOA-inspired embedding

Templates reduce boilerplate code and implement best practices.

In [ ]:
# Example 8: Using templates
from pennylane import templates

dev_4q = qml.device('default.qubit', wires=4)

@qml.qnode(dev_4q)
def template_circuit(features, weights):
    """
    Circuit using templates for data encoding and variational layers.
    
    features: Classical data to encode (shape: [4])
    weights: Trainable parameters (shape: [2, 4, 3])
    """
    # Encode classical data using angle embedding
    qml.AngleEmbedding(features, wires=range(4))
    
    # Apply strongly entangling layers
    qml.StronglyEntanglingLayers(weights, wires=range(4))
    
    # Measure first qubit
    return qml.expval(qml.PauliZ(0))

# Generate random features and weights
features = np.array([0.1, 0.5, 0.9, 0.3])
weights = np.random.randn(2, 4, 3)  # 2 layers, 4 wires, 3 params per wire

result = template_circuit(features, weights)
print(f"Template circuit output: {result:.4f}")

# Visualize the circuit
print("\nCircuit structure:")
print(qml.draw(template_circuit)(features, weights))

## 9. Advanced: Quantum Gradients and VQE Primer

The **Variational Quantum Eigensolver (VQE)** is a fundamental algorithm for quantum chemistry and optimization. It uses:
1. A parameterized quantum circuit (ansatz)
2. A classical optimizer
3. Measurement of an observable (Hamiltonian)

The goal is to find parameters that minimize the expectation value of the Hamiltonian, approximating the ground state energy.

Here's a simple example with a toy Hamiltonian:

In [ ]:
# Example 9: VQE-style optimization
# Define a simple Hamiltonian: H = 0.5*Z₀ + 0.3*Z₁ - 0.4*X₀X₁
coeffs = [0.5, 0.3, -0.4]
obs = [qml.PauliZ(0), qml.PauliZ(1), qml.PauliX(0) @ qml.PauliX(1)]
hamiltonian = qml.Hamiltonian(coeffs, obs)

print("Hamiltonian:")
print(hamiltonian)

@qml.qnode(dev)
def vqe_circuit(params):
    """Variational ansatz for VQE"""
    # Hardware-efficient ansatz
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=0)
    qml.RY(params[3], wires=1)
    
    # Measure Hamiltonian expectation
    return qml.expval(hamiltonian)

# Optimize to find ground state
opt = qml.AdamOptimizer(stepsize=0.1)
params = np.random.randn(4, requires_grad=True)

print("\nOptimizing for ground state energy...")
energies = []

for step in range(150):
    params, energy = opt.step_and_cost(vqe_circuit, params)
    energies.append(energy)
    
    if step % 30 == 0:
        print(f"Step {step:3d}: Energy = {energy:.6f}")

print(f"\nGround state energy (approx): {energy:.6f}")

# Visualize energy convergence
plt.figure(figsize=(10, 4))
plt.plot(energies, linewidth=2, color='green')
plt.xlabel('Optimization Step', fontsize=12)
plt.ylabel('Energy', fontsize=12)
plt.title('VQE Energy Convergence', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=min(energies), color='r', linestyle='--', alpha=0.5, label='Approximate ground state')
plt.legend()
plt.show()

## 10. Summary and Next Steps

### Key Takeaways:

1. **Devices**: Define where computations run (simulator or hardware)
2. **QNodes**: Wrap quantum functions and enable differentiation
3. **Gates**: Building blocks for quantum circuits
4. **Measurements**: Extract classical information from quantum states
5. **Differentiation**: Automatic gradient computation for optimization
6. **Optimization**: Built-in optimizers for training quantum circuits
7. **Templates**: Pre-built circuit architectures for common tasks

